# Transforming Target
One of the common requirements is to transform the target variable itself. 
hcrystalball implements ``TargetTransformer`` transformer for such a cases, that internally scales input data and return un-scaled again.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from hcrystalball.utils import get_sales_data

df = get_sales_data(n_dates=100, 
                    n_assortments=1, 
                    n_states=1, 
                    n_stores=1)
X, y = pd.DataFrame(index=df.index), df['Sales']

In [ ]:
from hcrystalball.wrappers import get_sklearn_wrapper
from hcrystalball.compose import TSColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from hcrystalball.preprocessing import TargetTransformer

## With Wrapper

In [ ]:
rf_model = get_sklearn_wrapper(RandomForestRegressor, random_state=42)
scaled = TargetTransformer(rf_model, StandardScaler())

In [ ]:
preds = (scaled.fit(X[:-10], y[:-10])
         .predict(X[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50)    
)
preds.plot(title=f"MAE:{(preds['Sales']-preds['sklearn']).abs().mean().round(3)}");

## With Pipeline

In [ ]:
X['trend'] = np.arange(len(X))

In [ ]:
# add standard scaler to the trend and let the date column pass in raw form
preprocessing = TSColumnTransformer(
    transformers=[        
        ('scaler', StandardScaler(), ['trend'])
    ])    
# define random forest model
rf_model = get_sklearn_wrapper(RandomForestRegressor, random_state=42)
# glue it together
sklearn_model_pipeline = Pipeline([
    ('preprocessing', preprocessing), 
    ('model', rf_model)
]) 

In [ ]:
scaled_pipeline = TargetTransformer(sklearn_model_pipeline, StandardScaler())

In [ ]:
preds = (scaled_pipeline.fit(X[:-10], y[:-10])
         .predict(X[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50)    
)
preds.plot(title=f"MAE:{(preds['Sales']-preds['sklearn']).abs().mean().round(3)}");